<a href="https://colab.research.google.com/github/shadilytn/Autonomy/blob/main/Microsoft_Learn_Scrap_with_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping, Processing, and Embedding

This Google Colab notebook demonstrates a complete workflow for extracting information from a website, processing it, and preparing it for use in applications like question answering or semantic search.

Here's a breakdown of its functionality:

Web Scraping: It starts by crawling a specified website (in this case, learn.microsoft.com/en-us/) and extracting the text content from multiple pages.

Data Processing: The extracted text is then processed by splitting it into smaller, overlapping chunks. This makes the text more manageable for further analysis and embedding.

Embedding: Using a pre-trained Sentence Transformer model, each text chunk is converted into a numerical representation called an embedding.
Embeddings capture the semantic meaning of the text, allowing for comparisons of similarity between different chunks.
Saving: Finally, the processed data, including the original text chunks, their source URLs, and the newly created embeddings, is saved to a JSON file in your Google Drive for later use.
In essence, the notebook takes raw website text and transforms it into a structured, semantically rich dataset.

## Install necessary libraries

In [ ]:
!pip install -q ipywidgets google-colab python-docx pypdf pandas nltk sentence-transformers torch tqdm pyarrow httpx beautifulsoup4 datasets requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Web scraping and data extraction script
This script crawls a website and extracts text content from each page.


In [ ]:
# prompt: write a script to navigate to the link https://learn.microsoft.com/en-us/ and start web scrapping and data extraction automatically on every page must scrap and extract all data, 100% data

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

def is_valid(url):
    """Checks whether `url` is a valid URL."""
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except:
        return False

def get_all_website_links(url):
    """
    Returns all URLs that is found on `url` in which it belongs to the same website
    """
    urls = set()
    domain_name = urlparse(url).netloc
    try:
        soup = BeautifulSoup(requests.get(url).content, "html.parser")
        for a_tag in soup.findAll("a"):
            href = a_tag.attrs.get("href")
            if href == "" or href is None:
                continue
            href = urljoin(url, href)
            parsed_href = urlparse(href)
            href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
            if not is_valid(href):
                continue
            if parsed_href.netloc == domain_name:
                urls.add(href)
    except Exception as e:
        print(f"Error processing {url}: {e}")
    return urls

def scrape_page_data(url):
    """Scrapes all text content from a given URL."""
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract all text from the page
        text = soup.get_text(separator='\n', strip=True)
        return text
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

def crawl_website(start_url, max_pages=100):
    """Crawls a website and scrapes data from each page."""
    visited_urls = set()
    urls_to_visit = {start_url}
    scraped_data = {}

    while urls_to_visit and len(visited_urls) < max_pages:
        current_url = urls_to_visit.pop()
        if current_url in visited_urls:
            continue

        print(f"Visiting: {current_url}")
        visited_urls.add(current_url)

        # Scrape data
        data = scrape_page_data(current_url)
        if data:
            scraped_data[current_url] = data

        # Find new links
        new_links = get_all_website_links(current_url)
        for link in new_links:
            if link not in visited_urls:
                urls_to_visit.add(link)

    return scraped_data

# Start the crawling process
start_url = "https://learn.microsoft.com/en-us/"
all_scraped_data = crawl_website(start_url)

# You can now process the `all_scraped_data` dictionary
# For example, print the number of pages scraped and the data from one page:
print(f"\nScraped data from {len(all_scraped_data)} pages.")
if all_scraped_data:
    first_url = list(all_scraped_data.keys())[0]
    print(f"\nData from the first scraped page ({first_url}):")
    # print(all_scraped_data[first_url][:500]) # Print first 500 characters


Visiting: https://learn.microsoft.com/en-us/
Visiting: https://learn.microsoft.com/en-us/contribute


<ipython-input-7-51f493157538>:23: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  for a_tag in soup.findAll("a"):


Visiting: https://learn.microsoft.com/en-us/samples/browse/
Visiting: https://learn.microsoft.com/en-us/legal/termsofuse
Visiting: https://learn.microsoft.com/en-us/training/contributor/contribute-docs-browser/
Visiting: https://learn.microsoft.com/en-us/certifications/posts/microsoft-learn-why-get-certified
Visiting: https://learn.microsoft.com/en-us/answers/questions/
Visiting: https://learn.microsoft.com/en-us/credentials/certifications/power-platform-fundamentals/
Visiting: https://learn.microsoft.com/en-us/answers/tags/122/microsoft-authenticator/
Visiting: https://learn.microsoft.com/en-us/training/contributor/contribute-docs-browser/7-exercise-pull-request-sign-off
Visiting: https://learn.microsoft.com/en-us/credentials/certifications/schedule-through-pearson-vue
Visiting: https://learn.microsoft.com/en-us/answers/tags/122/microsoft-authenticator
Visiting: https://learn.microsoft.com/en-us/training/contributor/contribute-docs-browser/6-pull-request-overview
Visiting: https://lea

Visiting: https://learn.microsoft.com/en-us/system-center/
Visiting: https://learn.microsoft.com/en-us/sql/t-sql/statements/set-concat-null-yields-null-transact-sql
Visiting: https://learn.microsoft.com/en-us/azure/azure-sql/virtual-machines/windows/availability-group-overview
Visiting: https://learn.microsoft.com/en-us/sql/relational-databases/replication/publish/create-a-publication
Visiting: https://learn.microsoft.com/en-us/answers/tags/221/azure-virtual-desktop/
Visiting: https://learn.microsoft.com/en-us/dotnet/architecture/serverless/
Visiting: https://learn.microsoft.com/en-us/dynamics365/customer-insights/
Visiting: https://learn.microsoft.com/en-us/answers/questions/2277621/im-not-able-to-login-into-the-microsoft-account-cr
Visiting: https://learn.microsoft.com/en-us/contribute/dotnet/dotnet-contribute
Visiting: https://learn.microsoft.com/en-us/dotnet/ai/how-to/use-redis-for-memory

Scraped data from 100 pages.

Data from the first scraped page (https://learn.microsoft.com/e

## Data processing and embedding script
This script takes the scraped data, chunks it, and creates embeddings using a sentence transformer model.

In [ ]:
# prompt: write a script to convert, format, embed the full scrapped and extracted data to structured, embedded data chunks

import torch
from sentence_transformers import SentenceTransformer # Changed import
from datasets import Dataset
from tqdm.auto import tqdm

# Check for GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

def chunk_text(text, chunk_size=500, chunk_overlap=50):
    """Splits text into chunks with overlap."""
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i + chunk_size]
        chunks.append(" ".join(chunk))
        i += chunk_size - chunk_overlap
        if i >= len(words) - chunk_overlap and i < len(words): # Handle the last chunk
             chunks.append(" ".join(words[i:]))
             break

    return chunks

def process_scraped_data(scraped_data, chunk_size=500, chunk_overlap=50):
    """
    Converts scraped data into formatted chunks and embeds them.
    Returns a list of dictionaries, each containing chunk text, source URL, and embedding.
    """
    processed_chunks = []
    for url, text in tqdm(scraped_data.items(), desc="Processing scraped data"):
        if text:
            chunks = chunk_text(text, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
            for chunk in chunks:
                processed_chunks.append({
                    'text': chunk,
                    'source': url,
                })
    return processed_chunks

def embed_chunks(processed_chunks, model, batch_size=32):
    """Embeds the text chunks using the sentence transformer model."""
    # Extract texts for embedding
    texts_to_embed = [chunk['text'] for chunk in processed_chunks]

    # Create a Hugging Face Dataset
    dataset = Dataset.from_dict({'text': texts_to_embed})

    # Define a function to apply embeddings
    def get_embeddings(batch):
        return {'embedding': model.encode(batch['text'], convert_to_tensor=True).tolist()}

    # Apply the embedding function in batches
    dataset = dataset.map(get_embeddings, batched=True, batch_size=batch_size)

    # Update the original processed_chunks list with embeddings
    for i, item in enumerate(processed_chunks):
        item['embedding'] = dataset[i]['embedding']

    return processed_chunks

# --- Main script for processing and embedding ---

# Process the scraped data into chunks
formatted_chunks = process_scraped_data(all_scraped_data)

# Embed the chunks
embedded_data = embed_chunks(formatted_chunks, model)

# `embedded_data` is now a list of dictionaries, where each dictionary
# represents a chunk with its text, source URL, and embedding.
# You can now use this data for similarity search, indexing, etc.

print(f"\nCreated {len(embedded_data)} embedded chunks.")
if embedded_data:
    print("\nExample of an embedded chunk:")
embedded_data[0]

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Processing scraped data:   0%|          | 0/100 [00:00<?, ?it/s]

Map:   0%|          | 0/340 [00:00<?, ? examples/s]


Created 340 embedded chunks.

Example of an embedded chunk:


{'text': 'Microsoft Learn: Build skills that open doors in your career Skip to main content This browser is no longer supported. Upgrade to Microsoft Edge to take advantage of the latest features, security updates, and technical support. Download Microsoft Edge More info about Internet Explorer and Microsoft Edge Microsoft Learn. Spark possibility. Build skills that open doors. See all you can do with documentation, hands-on training, and certifications to help you get the most from Microsoft products. Search Learn by doing Gain the skills you can apply to everyday situations through hands-on training personalized to your needs, at your own pace or with our global network of learning partners. Take training Find technical documentation Get tools and step-by-step guidance to help you get the most from Microsoft products such as Azure, Windows, Office, Dynamics, Power Apps, Teams, and more. View documentation Showcase your skills Advance in your career by completing challenges that demon

## Save the embedded dataset to Google Drive
This script saves the processed and embedded data to a JSON file in your Google Drive.


In [ ]:
import json
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the output file path
output_file_path = '/content/drive/My Drive/Output/embedded_dataset.json'

# Ensure the output directory exists
import os
output_dir = os.path.dirname(output_file_path)
os.makedirs(output_dir, exist_ok=True)

# Save the embedded data to a JSON file
with open(output_file_path, 'w') as f:
    json.dump(embedded_data, f, indent=2)

print(f"\nSaved embedded dataset to: {output_file_path}")

Mounted at /content/drive

Saved embedded dataset to: /content/drive/My Drive/Output/embedded_dataset.json
